# FFF Workshop

## A4: Selections and interactions

### Outline

- Selecting sets of compounds and poses
- Working with dataframes
- Working with HIPPO subsites
- Interaction profiling and visualisation

## Selecting sets of compounds and poses

Below are some examples of how selections of compounds and poses can be created

In [ ]:
# setup the animal
%load_ext autoreload
%autoreload 2
# from mrich import print
import hippo
animal = hippo.HIPPO(
    "A71EV2A_demo",
    "../data/A71EV2A.sqlite",
)

In [ ]:
# by tag
soaks = animal.compounds(tag="soaks")
print(soaks)

# by ids
cset = animal.compounds[1,3,4,6,12,43]
print("compounds with ID's 1,3,4,6,12,43:", cset)

# from set of poses
hit_compounds = animal.poses(tag="hits").compounds
print("hit compounds:", hit_compounds)

# by metadata info
double_soaks = hit_compounds.get_by_metadata(key="SoakDB count", value=2)
print("double soaks:", double_soaks)

Some boolean operations are also supported:

In [ ]:
# add two sets:
triple_soaks = hit_compounds.get_by_metadata(key="SoakDB count", value=3)
selection = double_soaks + triple_soaks
print("double and triple soaks:", selection)

# subtract:
print("hit_compounds - double_soaks:", hit_compounds - double_soaks)

# single addition:
print("double_soaks + C3:", double_soaks + animal.C3)

# set intersection
print("hit_compounds & double_soaks:", hit_compounds & double_soaks)

# set union
print("hit_compounds | double_soaks:", hit_compounds | double_soaks)

# set exclusive OR
print("hit_compounds ^ double_soaks:", hit_compounds ^ double_soaks)

Once you are happy with a selection you can add a tag for easier future retrieval:

In [ ]:
selection.add_tag("double and triple soaks")

See also the API reference documentation:

- [CompoundTable](https://hippo-docs.winokan.com/en/latest/compounds.html#hippo.cset.CompoundTable) (i.e. `animal.compounds`)
- [CompoundSet](https://hippo-docs.winokan.com/en/latest/compounds.html#hippo.cset.CompoundSet) (i.e. a set of compounds)
- [PoseTable](https://hippo-docs.winokan.com/en/latest/poses.html#hippo.pset.PoseTable) (i.e. `animal.poses`)
- [PoseSet](https://hippo-docs.winokan.com/en/latest/poses.html#hippo.pset.PoseSet) (i.e. a set of poses)

## Working with DataFrames

`pandas.DataFrame` objects are a popular way to work with tabular data, and if you are familiar with it you can get DataFrame representations of HIPPO objects as well:

In [ ]:
df = selection.get_df()
df.head()

In [ ]:
# more options are available to add more columns:
df = selection.get_df(
    smiles=True,
    inchikey=True,
    alias=True,
    metadata=True,
    expand_metadata=True,
    num_poses=True,
)
df.head()

See also the API reference:

- [CompoundSet.get_df()](https://hippo-docs.winokan.com/en/latest/compounds.html#hippo.cset.CompoundSet.get_df)
- [PoseSet.get_df()](https://hippo-docs.winokan.com/en/latest/poses.html#hippo.pset.PoseSet.get_df)

Or try the `help()` function:

In [ ]:
help(selection.get_df)

Once you have a dataframe you can perform all kinds of filtering operations and also add columns from other sources

In [ ]:
# select compounds with multiple poses
filtered_df = df[df["num_poses"] > 1]

# sort compounds by SoakDB count
sorted_df = filtered_df.sort_values(by="SoakDB count", ascending=False)

sorted_df.head()

You can then get back to a HIPPO `CompoundSet`:

In [ ]:
# simplest way:
cset = animal.compounds[sorted_df.index]

# to maintain custom ordering:
cset = animal.compounds(ids=list(sorted_df.index), sort=False)

cset.interactive()

## Working with HIPPO subsites

HIPPO subsite objects are an additional way to group `Pose` objects by arbitrary 'pockets' or 'subsites'.

They can be assigned from Fragalysis'/XCA Canonical sites for example:

In [ ]:
hits = animal.poses(tag="hits")
hits.set_subsites_from_metadata_field(field="CanonSites alias")
hits.subsite_summary();

Then you can also select by subsites:

In [ ]:
active_site_hits = animal.poses(subsite=1).get_by_tag("hits")
print("active_site_hits:", active_site_hits)

If you have poses which were inspired by other poses (more on this later), you can propogate through the site assignments with:

`animal.db.set_derivative_subsites()`

Manually inserting or assigning subsites is not broadly recommended but you can do it with:

- `animal.db.insert_subsite()` [docs](https://hippo-docs.winokan.com/en/latest/db.html#hippo.db.Database.insert_subsite)
- `animal.db.insert_subsite_tag()` [docs](https://hippo-docs.winokan.com/en/latest/db.html#hippo.db.Database.insert_subsite_tag)

## Interaction profiling and visualisation

Interactions can be profiled directly from `Pose` objects. This can take some time so it's best to work with a subset of poses.

In [ ]:
import mrich # this is just for the progress bar

active_site_fragments = animal.poses(tag="[Other] Active site fragment")

for pose in mrich.track(active_site_fragments):
    pose.calculate_interactions()

Once the interactions are in the database can be represented as an `InteractionSet` object ([docs](https://hippo-docs.winokan.com/en/latest/interactions.html#hippo.iset.InteractionSet)):

In [ ]:
# from a PoseSet
active_site_fragments_iset = active_site_fragments.interactions
print("active_site_fragments_iset:", active_site_fragments_iset)

# from a single Pose
pose = active_site_fragments[1]
pose_iset = pose.interactions
print("pose_iset:", pose_iset)

In [ ]:
pose_iset.summary()

In [ ]:
# see a 3D render with interactions
pose.render(interactions=True)

In [ ]:
# access a dataframe of interactions
pose.interactions.df

Interactions can be visualised in a few other ways:

In [ ]:
fig = animal.plot_interaction_punchcard()
fig

This is a `plotly.graph_objects.Figure()` object that can be manipulated and written to files.

I recommend using the `molparse.write` method:

In [ ]:
import molparse as mp

fig = fig.update_layout(title="A71EV2A Active Site Fragment Interactions")

mp.write("punchcard.html", fig)
mp.write("punchcard.pdf", fig, width=1200, height=600)

fig